# Change Tf and keras version because that's what the layer was written in

In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.1.2

# Import necessary libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
import sys
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import cv2
from sklearn.utils import shuffle
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Defining the layer
This has been taken from https://software.intel.com/content/www/us/en/develop/articles/keras-implementation-of-siamese-like-networks.html

In [ ]:
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Flatten
from keras.models import Model, Sequential
from keras.engine import InputSpec, Layer
from keras import regularizers
from keras.optimizers import SGD, Adam
from keras.utils.conv_utils import conv_output_length
from keras import activations
import numpy as np


class Normalized_Correlation_Layer(Layer):

# create a class inherited from keras.engine.Layer.

    def __init__(self, patch_size=(5, 5),
          dim_ordering='tf',
          border_mode='same',
          stride=(1, 1),
          activation=None,
          **kwargs):

       if border_mode != 'same': 
          raise ValueError('Invalid border mode for Correlation Layer ' 
                     '(only "same" is supported as of now):', border_mode) 
       self.kernel_size = patch_size 
       self.subsample = stride 
       self.dim_ordering = dim_ordering 
       self.border_mode = border_mode 
       self.activation = activations.get(activation) 
       super(Normalized_Correlation_Layer, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        return(input_shape[0][0], input_shape[0][1], input_shape[0][2], self.kernel_size[0] * input_shape[0][2]*input_shape[0][-1])

    def get_config(self): 
        config = {'patch_size': self.kernel_size, 
              'activation': self.activation.__name__, 
              'border_mode': self.border_mode, 
              'stride': self.subsample, 
              'dim_ordering': self.dim_ordering} 
        base_config = super(Correlation_Layer, self).get_config() 
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, x, mask=None):
        input_1, input_2 = x
        stride_row, stride_col = self.subsample
        inp_shape = input_1._keras_shape
        output_shape = self.compute_output_shape([inp_shape, inp_shape])
        padding_row = (int(self.kernel_size[0] / 2),int(self.kernel_size[0] / 2)) 
        padding_col = (int(self.kernel_size[1] / 2),int(self.kernel_size[1] / 2)) 
        input_1 = K.spatial_2d_padding(input_1, padding =(padding_row,padding_col)) 
        input_2 = K.spatial_2d_padding(input_2, padding = ((padding_row[0]*2, padding_row[1]*2),padding_col)) 
        output_row = output_shape[1] 
        output_col = output_shape[2] 
        output = [] 
        for k in range(inp_shape[-1]):
               xc_1 = [] 
               xc_2 = [] 
            #    print("here")
               for i in range(padding_row[0]): 
                  for j in range(output_col): 
                     xc_2.append(K.reshape(input_2[:, i:i+self.kernel_size[0], j:j+self.kernel_size[1], k], 
                                 (-1, 1,self.kernel_size[0]*self.kernel_size[1])))

        for i in range(output_row): 
            slice_row = slice(i, i + self.kernel_size[0]) 
            slice_row2 = slice(i + padding_row[0], i +self.kernel_size[0] + padding_row[0]) 
            # print("dfg")
            for j in range(output_col): 
                slice_col = slice(j, j + self.kernel_size[1]) 
                xc_2.append(K.reshape(input_2[:, slice_row2, slice_col, k], 
                           (-1, 1,self.kernel_size[0]*self.kernel_size[1]))) 
                xc_1.append(K.reshape(input_1[:, slice_row, slice_col, k], 
                             (-1, 1,self.kernel_size[0]*self.kernel_size[1])))
        for i in range(output_row, output_row+padding_row[1]): 
            for j in range(output_col): 
                xc_2.append(K.reshape(input_2[:, i:i+ self.kernel_size[0], j:j+self.kernel_size[1], k], 
                           (-1, 1,self.kernel_size[0]*self.kernel_size[1]))) 

        xc_1_aggregate = K.concatenate(xc_1, axis=1)
        xc_1_mean = K.mean(xc_1_aggregate, axis=-1, keepdims=True) 
        xc_1_std = K.std(xc_1_aggregate, axis=-1, keepdims=True) 
        xc_1_aggregate = (xc_1_aggregate - xc_1_mean) / xc_1_std

        xc_2_aggregate = K.concatenate(xc_2, axis=1)
        xc_2_mean = K.mean(xc_2_aggregate, axis=-1, keepdims=True) 
        xc_2_std = K.std(xc_2_aggregate, axis=-1, keepdims=True) 
        xc_2_aggregate = (xc_2_aggregate - xc_2_mean) / xc_2_std

        xc_1_aggregate = K.permute_dimensions(xc_1_aggregate, (0, 2, 1)) 
        block = [] 
        len_xc_1= len(xc_1) 
        print("asdf")
        for i in range(len_xc_1):
                 #This for loop is to compute the product of a given patch of feature map 1 and the feature maps on which it is supposed to
            sl1 = slice(int(i/inp_shape[2])*inp_shape[2], 
                  int(i/inp_shape[2])*inp_shape[2]+inp_shape[2]*self.kernel_size[0]) 
                      #This calculates which are the patches of feature map 2 to be considered for a given patch of first feature map.

            block.append(K.reshape(K.batch_dot(xc_2_aggregate[:,sl1,:], 
                          xc_1_aggregate[:,:,i]),(-1,1,1,inp_shape[2] *self.kernel_size[0]))) 

        block = K.concatenate(block, axis=1) 
        # print("zxcv")
        block= K.reshape(block,(-1,output_row,output_col,inp_shape[2] *self.kernel_size[0])) 
        output.append(block)

        output = self.activation(output) 
        try:
            print(output.numpy().shape)
        except:
            pass
        return output


In [ ]:
!ls /kaggle/input/abstraction-and-reasoning-challenge

## Load data and save to dictionaries

In [ ]:
os.chdir('/kaggle/input/abstraction-and-reasoning-challenge')
full_data = []
def read_files(dir_):
    file_data = {}
    file_data_train = []
    file_names = []
    for file in list(os.listdir(dir_)):
        with open(dir_ + '/' + file) as f:

            fd = json.loads(f.read())
            if dir_ != 'test':
                file_data[file] = {
                				 "train":{ "ip":[i['input'] for i in fd['train']], 
                				 		   "op": [i['output'] for i in fd['train']]},

                				 "test":{ "ip":[i['input'] for i in fd['test']], 
                				 		  "op": [i['output'] for i in fd['test']]}
                				 }
                file_names.append(file)
            else:
                file_data[file] = {
                				 "train":{ "ip":[i['input'] for i in fd['train']], 
                				 			"op": [i['output'] for i in fd['train']]},

                				 "test":{"ip":[i['input'] for i in fd['test']], 
                				 "op": []}
                				 }
                file_names.append(file)
    full_data.append(file_data)

    with open('/kaggle/working'+dir_ + '_file_data2.pickle', 'wb') as f:
        pickle.dump(file_data, f)
    with open('/kaggle/working'+dir_ + '_data2.pickle', 'wb') as f:
        pickle.dump([file_data, file_names], f)

    return file_data, file_names


for i in ['training','test','evaluation']:
    if os.path.isdir(i):
        x = read_files(i)


In [ ]:
train_dict, val_dict, test_dict = full_data

In [ ]:
# print full arrays without ... in between
def fullprint(*args, **kwargs):
  from pprint import pprint
  import numpy
  opt = numpy.get_printoptions()
  numpy.set_printoptions(threshold=numpy.inf)
  pprint(*args, **kwargs)
  numpy.set_printoptions(**opt)

Padding all arrays to keep the sizes same (50x50)

In [ ]:
pad_arr = np.zeros((50,50),dtype=np.int8) 

## Superimpose inputs and outputs to the center of padding array
Original values are increased by 1 because the padding is zero and so all original zero values should be maintained

In [ ]:
def center_merge(a,b):
#     print(a.shape,b.shape)
    x1 = a.shape[1]//2 - b.shape[1]//2
    y1 = a.shape[0]//2 - b.shape[0]//2
    x2 = a.shape[1]//2 - b.shape[1]//2 + b.shape[1]
    y2 = a.shape[0]//2 - b.shape[0]//2 + b.shape[0]
#     try:
    a[y1:y2, x1:x2] = b+1   # b+1 because the padding is zero and so all original zero values should be maintained
#     except:
    a = np.reshape(a, (50,50))
    
    return a

In [ ]:
# Pad all arrays in train, validation and test set
for dicts in [train_dict, val_dict, test_dict]:
    for dct in dicts:
        for setval in dicts[dct].keys():
            for iop in dicts[dct][setval].keys():
                for i,mat in enumerate(dicts[dct][setval][iop]):
                    dicts[dct][setval][iop][i] = center_merge(pad_arr.copy(),np.array(mat))

# Changing colours in inputs and outputs 
This generates permutations for mapping original colour to other colours in the range

In [ ]:
def create_mappings():
    l = np.array([i for i in range(2,12)])
    shuffled = []
    for i in range(50):
        sh = l.copy()
        np.random.shuffle(sh)
        shuffled.append(sh)
    mappings = [dict(zip(l,i)) for i in shuffled]
    return mappings

# mappings = create_mappings()

def create_permutations(a,dicts):
    permuted = []
    for d in dicts:
        b = a.copy()
        for k, v in d.items():
            b[a==k] = v
        b = np.array(b)
        permuted.append(np.reshape(np.array(b).astype(np.uint8),(50,50)))
    permuted = np.reshape(np.array(permuted),(-1,50,50))
    return permuted

## I previously intended to create masks of nxm dimension and input those as different channels after convolving
Takes a lot of memory to do so, so it is not used further

In [ ]:
def create_masks(orig):
    kernels = [np.ones((1,1),dtype=np.uint8),np.ones((2,1),dtype=np.uint8),
               np.ones((1,2),dtype=np.uint8),np.ones((3,1),dtype=np.uint8),
               np.ones((1,3),dtype=np.uint8),np.ones((3,3),dtype=np.uint8)]
    arr = orig.copy()
    masked = np.array(arr,dtype=np.uint8)
    for kernel in kernels:
        mask = np.zeros((50,50),dtype=np.uint8)
        mask[:kernel.shape[0],:kernel.shape[1]] = kernel
        mask_orig = mask.copy()
        mask = np.roll(mask,1)
        print(mask.all()!=mask_orig.all())
        counter=0
        while not mask.all()!=mask_orig.all():
            mask = np.roll(mask,1)
            print(counter)
            counter+=1
            arr_mask = np.bitwise_and(arr,mask)
            if arr_mask.all() == np.zeros((50,50)).all():
                continue
            masked = np.append(masked,arr_mask, axis=0)
            if mask[-1,-1]==1:
                print(counter,'\n',mask)
                break
    return masked
    

# Convert all inputs/outputs to onehot maps, ie one channel for every value in 0-11

In [ ]:
def expand_op(op):
    op = np.reshape(np.array(op),(50,50))
    op_arr = []
    l = np.array([i for i in range(0,12)])
    for i in l:
        op_arr.append(np.reshape(np.array([op==i]).astype(np.uint8), (50,50)))
    return np.reshape(np.array(op_arr).astype(np.uint8), (50,50,12))

# Apply all transformations

In [ ]:
def generate_data(data_dict):
    for task in sorted(list(data_dict.keys())):
        ip_perm = []
        op_perm = []
#         print(np.array(data_dict[task]['train']['ip']).shape)
        for i in range(len(data_dict[task]['train']['ip'])):
            mappings = create_mappings()
            
            ip_perm.extend(create_permutations(data_dict[task]['train']['ip'][i],mappings))
            op_perm.extend(create_permutations(data_dict[task]['train']['op'][i],mappings))
        data_dict[task]['train']['ip'].extend(ip_perm)
#         print(np.array(data_dict[task]['train']['ip']).shape)
        data_dict[task]['train']['op'].extend(op_perm)
        data_dict[task]['train']['ip'],data_dict[task]['train']['op'] = shuffle(data_dict[task]['train']['ip'],data_dict[task]['train']['op'],random_state=69)
        
        data_dict[task]['train']['ip'] = [expand_op(i) for i in data_dict[task]['train']['ip']]
        data_dict[task]['train']['op'] = [expand_op(i) for i in data_dict[task]['train']['op']]
#         np.random.shuffle(data_dict[task]['train']['ip'], seed=69)
#         np.random.shuffle(data_dict[task]['train']['op'], seed=69)
    return data_dict
        

In [ ]:
train_dict = generate_data(train_dict)
# val_dict = generate_data(val_dict)
# test_dict = generate_data(test_dict)

# The model:
## The model consumes a lot of memory so only a few layers are kept

In [ ]:
dt = 'float32'
def create_model():
    ip = keras.layers.Input((50,50, 12))
    ncx1_1 =  Normalized_Correlation_Layer(patch_size=(1, 1))([ip,ip])
    ncx1_1 = keras.layers.Reshape((50,50,600))(ncx1_1)
    ncn1_1 = keras.layers.Conv2D(64, (1,1), activation = 'relu', dtype=dt)(ip)
    ncn2_1 = keras.layers.Conv2D(64, (1,1), activation = 'relu', dtype=dt)(ncx1_1)
    ncx2_1 =  Normalized_Correlation_Layer(patch_size=(1, 1),dtype=dt)([ncn1_1,ncn2_1])
    ncn3 = keras.layers.LocallyConnected2D(filters=64,kernel_size=(1,1), activation = 'relu', dtype=dt)(ncx2_1)
    ncn4 = keras.layers.Conv2D(12, (1,1), activation = 'sigmoid', dtype=dt)(ncn3)
    
    model = keras.models.Model(ip,ncn4)
    
    return model

In [ ]:
md = create_model()

In [ ]:
md.summary()

In [ ]:
md.compile(optimizer = 'adam', loss = 'categorical_crossentropy')

## A custom generator that generates data as per task, keeping all tasks in same batch

In [ ]:
class Custom_Generator(keras.utils.Sequence) :
    def __init__(self, data_dict, batch_size=50, prps='train') :
        self.data_dict = data_dict
        self.tg = self.train_generator()
        self.prps = prps
    def __len__(self) :
        x = 0
        for task in sorted(list(self.data_dict.keys())):
            x += len(self.data_dict[task][self.prps]['ip'])
        print("len = ",np.ceil(x/50).astype(np.int))
        return np.ceil(x/50).astype(np.int)
    
    def train_generator(self):
        for task in sorted(list(self.data_dict.keys())):
            x = np.array(self.data_dict[task][self.prps]['ip'])
            x = np.reshape(x, (-1,50,50,12))
            y = self.data_dict[task][self.prps]['op']
            y = np.reshape(y, (-1,50,50,12))
#             print(x.shape)
            for i in range(0,len(x),50):
                yield (np.array([x[i:i+50]]), np.array([y[i:i+50]]))
            
    def __getitem__(self, idx) :
        
        batch_x, batch_y = next(self.tg)
        print(idx, batch_x.shape,batch_y.shape)
        return ( np.reshape(batch_x,(-1,50,50,12)), np.reshape(batch_y,(-1,50,50,12)) )



In [ ]:
train_gen = Custom_Generator(train_dict)

## Does not fit due to memory constraints, speed/no of operations etc.

In [ ]:
md.fit_generator(train_gen,verbose=1)

In [ ]:
# from numpy import copy, random, arange

# random.seed(0)
# data = random.randint(30, size=10**5)

# d = {4: 0, 9: 5, 14: 10, 19: 15, 20: 0, 21: 1, 22: 2, 23: 3, 24: 0}
# dk = d.keys()
# dv = d.values()

# def f1(a, d):
#     b = copy(a)
#     for k, v in d.items():
#         b[a==k] = v
#     return b

# def f2(a, d):
#     for i in range(len(a)):
#         a[i] = d.get(a[i], a[i])
#     return a

# dget = np.vectorize(d.get)
# def f4(a,d):
#     return dget(a)


# a = copy(data)
# res = f2(a, d)

# # assert (f1(data, d) == res).all()
# # assert (f3(data, dk, dv) == res).all()

In [ ]:
# %timeit f1(data,d)

In [ ]:
# %timeit f4(data,d)